In [8]:
import numpy as np 
import matplotlib.pyplot as plt
import scienceplots
import matplotlib.animation as animation
from scipy.sparse import dok_matrix
import multiprocessing as mp
import scipy as sp
import matplotlib.ticker as ticker
import project

In [9]:
#General plot parameters and size definition
plt.style.use('science')
plt.rcParams["figure.figsize"] = (12,7)
plt.rcParams.update({"axes.grid" : True})
plt.rcParams.update({"axes.titlesize": 18})
plt.rcParams.update({"axes.labelsize": 15})

In [10]:
def spikelet_fun(k, num):
    return (k * 1.76269962 * (num*num)*0.150656197 + 68.24441504*k)

In [11]:
#loading up the parameters

#params_HH = project.utils.load_HH()
params_LIF = project.utils.load_LIF()
#params_IZH = project.utils.load_ISH()
#params_ML = project.utils.load_ML()

#time parameters
t_final = 5000
dt = 0.01



#Other constant parameters
num_neurons = 5
#Variation for the initial conditions
volt_var = np.random.normal(0,5,num_neurons)
#rec_var = np.random.normal(0,1,num_neurons)
Isyn = 0.0
tau = 0.65
#V0 = -65 * np.ones(num_neurons) + volt_var
y0 = -65 * np.ones(num_neurons) + volt_var
#w0 = 0 * np.ones(num_neurons) + rec_var
#u0 = 0 * np.ones(num_neurons) + rec_var
#n0 = 0 * np.ones(num_neurons) + rec_var
#m0 = 0 * np.ones(num_neurons) + rec_var
#h0 = 0 * np.ones(num_neurons) + rec_var
order = 2


#fixing random seed
np.random.seed(1234)


    #I_HH[:,i] = 2.5 
    #I_IF[:,i] = 2.5 
    #I_IZH[:,i] = 2.5
    #I_ML[:,i] = 2.5 

#HH_gap_junction = np.zeros((int(t_final/dt),num_neurons))
#HH_synaptic_current = np.zeros((int(t_final/dt),num_neurons))

IF_gap_junction = np.zeros((int(t_final/dt),num_neurons))
IF_synaptic_current = np.zeros((int(t_final/dt),num_neurons))

#IZH_gap_junction = np.zeros((int(t_final/dt),num_neurons))
#IZH_synaptic_current = np.zeros((int(t_final/dt),num_neurons))

#ML_gap_junction = np.zeros((int(t_final/dt),num_neurons))
#ML_synaptic_current = np.zeros((int(t_final/dt),num_neurons))

In [12]:
#Creating the Electrical and Chemical connectivity matrices, in this case for 10 neurons
E_matrix,seed, ratio = project.utils.create_matrix('random',num_neurons,synapse_type='electrical',connection_fraction=1)
C_matrix, seed, ratio = project.utils.create_matrix('random',num_neurons,connection_fraction=1)

In [13]:
k = np.linspace(0.05,0.5,10)
#gap_current_HH = np.zeros_like(k)
#gap_current_mean_HH = np.zeros_like(k)
#gap_current_variance_HH = np.zeros_like(k)
#noise_variance_HH = np.zeros_like(k)
#chi_HH = np.zeros_like(k)
#rel_HH = np.zeros_like(k)
gap_current_IF = np.zeros_like(k)
gap_current_mean_IF = np.zeros_like(k)
gap_current_variance_IF = np.zeros_like(k)
noise_variance_IF = np.zeros_like(k)
chi_IF = np.zeros_like(k)
rel_IF = np.zeros_like(k)
#gap_current_IZH = np.zeros_like(k)
#gap_current_mean_IZH = np.zeros_like(k)
#gap_current_variance_IZH = np.zeros_like(k)
#noise_variance_IZH = np.zeros_like(k)
#chi_IZH = np.zeros_like(k)
#rel_IZH = np.zeros_like(k)
#gap_current_ML = np.zeros_like(k)
#gap_current_mean_ML = np.zeros_like(k)
#gap_current_variance_ML = np.zeros_like(k)
#noise_variance_ML = np.zeros_like(k)
#chi_ML = np.zeros_like(k)
#rel_ML = np.zeros_like(k)
t_R = 0.5
t = np.linspace(0,t_final,int(t_final/dt))
for i in range(len(k)):
    #generating the intensities
    #I_HH = np.zeros((int(t_final/dt),num_neurons))
    I_IF = np.zeros((int(t_final/dt),num_neurons))
    #I_IZH = np.zeros((int(t_final/dt),num_neurons))
    #I_ML = np.zeros((int(t_final/dt),num_neurons))

    for j in range(0,num_neurons):
        x_normal = np.random.normal(0,k[i],int(t_final/dt))
        x_if = np.random.normal(0,k[i],int(t_final/dt))
        #I_HH[:,j] = 2.5 * (1 + 2*x_normal)
        I_IF[:,j] = 2.5 * (1 + 2*x_if)
        #I_IZH[:,j] = 2.5 * (1 + 2*x_normal)
        #I_ML[:,j] = 2.5 * (1+ 2*x_normal)
    #dataHH_adjust, completeHH_adjust, matrixHH_adjust, synaptic_HH = project.models.HH_Neuron_Network_tests(dt,t_final,order,V0,n0,m0,h0,*params_HH,I_HH,Isyn,0.01,tau,E_matrix,C_matrix,HH_gap_junction,HH_synaptic_current)
    dataIF_adjust, completeIF_adjust, matrixIF_adjust, synaptic_IF = project.models.LIF_Neuron_Network_tests(dt, t_final,order,y0,*params_LIF,I_IF,Isyn,0.003,tau,spikelet_fun(0.003,5),E_matrix,C_matrix, IF_gap_junction, IF_synaptic_current)
    #dataIZH_adjust, completeIZH_adjust, matrixIZH_adjust, synaptic_IZH = project.models.IZH_Neuron_Network_tests(dt,t_final,order,y0,u0,I_IZH,Isyn,*params_IZH,0.2,tau,E_matrix,C_matrix,IZH_gap_junction,IZH_synaptic_current)
    #dataML_adjust, completeML_adjust, matrixML_adjust, synaptic_ML = project.models.ML_Neuron_Network_tests(dt,t_final,2,y0,w0,*params_ML,I_ML,Isyn,0.01,tau,E_matrix,C_matrix, ML_gap_junction, ML_synaptic_current)

    #gap_current_HH[i] = np.mean(HH_gap_junction[:,0])
    #gap_current_mean_HH[i] = np.mean(np.mean(HH_gap_junction,axis=1))
    #gap_current_variance_HH[i] = np.mean(np.var(HH_gap_junction,axis=1))
    #noise_variance_HH[i] = np.mean(np.var(I_HH,axis=1))
    #chi_HH[i] = project.utils.compute_chi(dataHH_adjust.T)
    #rel_HH[i], a = project.utils.compute_Reliability(matrixHH_adjust,t,t_R,num_neurons)

    gap_current_IF[i] = np.mean(IF_gap_junction[:,0])
    gap_current_mean_IF[i] = np.mean(np.mean(np.abs(IF_gap_junction),axis=1))
    gap_current_variance_IF[i] = np.mean(np.var(IF_gap_junction,axis=1))
    noise_variance_IF[i] = np.mean(np.var(I_IF,axis=1))
    chi_IF[i] = project.utils.compute_chi(dataIF_adjust.T)
    rel_IF[i], a = project.utils.compute_Reliability(matrixIF_adjust,t,t_R,num_neurons)

    #gap_current_IZH[i] = np.mean(IZH_gap_junction[:,0])
    #gap_current_mean_IZH[i] = np.mean(np.mean(np.abs(IZH_gap_junction),axis=1))
    #gap_current_variance_IZH[i] = np.mean(np.var(IZH_gap_junction,axis=1))
    #noise_variance_IZH[i] = np.mean(np.var(I_IZH,axis=1))
    #chi_IZH[i] = project.utils.compute_chi(dataIZH_adjust.T)
    #rel_IZH[i] , a= project.utils.compute_Reliability(matrixIZH_adjust,t,t_R,num_neurons)

    #gap_current_ML[i] = np.mean(ML_gap_junction[:,0])
    #gap_current_mean_ML[i] = np.mean(np.mean(np.abs(ML_gap_junction),axis=1))
    #gap_current_variance_ML[i] = np.mean(np.var(ML_gap_junction,axis=1))
    #noise_variance_ML[i] = np.mean(np.var(I_ML,axis=1))
    #chi_ML[i] = project.utils.compute_chi(dataML_adjust.T)
    #rel_ML[i], a = project.utils.compute_Reliability(matrixML_adjust,t,t_R,num_neurons)

In [14]:
#np.savetxt('gap_current_noise/gap_current_HH.txt',gap_current_HH)
#np.savetxt('gap_current_noise/gap_current_mean_HH.txt',gap_current_mean_HH)
#np.savetxt('gap_current_noise/gap_current_variance_HH.txt',gap_current_variance_HH)
#np.savetxt('gap_current_noise/noise_variance_HH.txt',noise_variance_HH)
#np.savetxt('gap_current_noise/chi_HH.txt',chi_HH)
#np.savetxt('gap_current_noise/rel_HH.txt',rel_HH)
np.savetxt('gap_current_noise2/gap_current_IF.txt',gap_current_IF)
np.savetxt('gap_current_noise2/gap_current_mean_IF.txt',gap_current_mean_IF)
np.savetxt('gap_current_noise2/gap_current_variance_IF.txt',gap_current_variance_IF)
np.savetxt('gap_current_noise2/noise_variance_IF.txt',noise_variance_IF)
np.savetxt('gap_current_noise2/chi_IF.txt',chi_IF)
np.savetxt('gap_current_noise2/rel_IF.txt',rel_IF)
#np.savetxt('gap_current_noise/gap_current_IZH.txt',gap_current_IZH)
#np.savetxt('gap_current_noise/gap_current_mean_IZH.txt',gap_current_mean_IZH)
#np.savetxt('gap_current_noise/gap_current_variance_IZH.txt',gap_current_variance_IZH)
#np.savetxt('gap_current_noise/noise_variance_IZH.txt',noise_variance_IZH)
#np.savetxt('gap_current_noise/chi_IZH.txt',chi_IZH)
#np.savetxt('gap_current_noise/rel_IZH.txt',rel_IZH)
#np.savetxt('gap_current_noise/gap_current_ML.txt',gap_current_ML)
#np.savetxt('gap_current_noise/gap_current_mean_ML.txt',gap_current_mean_ML)
#np.savetxt('gap_current_noise/gap_current_variance_ML.txt',gap_current_variance_ML)
#np.savetxt('gap_current_noise/noise_variance_ML.txt',noise_variance_ML)
#np.savetxt('gap_current_noise/chi_ML.txt',chi_ML)
#np.savetxt('gap_current_noise/rel_ML.txt',rel_ML)